<a href="https://colab.research.google.com/github/standenisa/Asistent-Ai/blob/main/IA_lab_4_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laborator 4: Naive Bayes
Spre deosebire de algoritmii de regresie logistică sau cei care învață să asocieze direct spațiul caracteristicilor de intrare cu etichete (algoritmi denumiți discriminanți), Naive Bayes este un algoritm de învățare generativ. Acesta modelează distribuțiile $p(x|y)$ și $p(y)$. Dacă variabila $y$ indică prin valorile discrete $0$ sau $1$ două clase distincte, atunci $p(x|y=0)$ modelează distribuția caracteristicilor unei clase, iar $p(x|y=1)$ modelează distribuția celeilalte.

Naive Bayes presupune faptul că variabilele de intrare sunt independente condițional, dându-se clasa țintă. Spre deosebire de algoritmii prezentați în laboratoarele anterioare, acest algoritm nu necesită aproximarea iterativă a parametrilor modelului, ci antrenarea se face in timp liniar prin evaluarea unei expresii.

Clasificatorul Naive Bayes se bazează pe teorema lui Bayes:

$$P(y|x) = \frac{P(x|y)P(y)}{P(x)}$$
unde $P(y|x)$ este probabilitatea lui $y$ dându-se $x$ (denumită probabilitatea posterioară), $P(x|y)$ este probabilitatea lui $x$ dându-se $y$ ( denumită likelihood), iar $P(y)$ și $P(x)$ sunt probabilitățile de a observa $y$, respectiv $x$, necondiționate. Ele se numesc probabilitatea apriori și probabilitatea marginală.


# Clasificatorul Naive Bayes - clasificarea e-mailurilor

În continuare se va realiza un clasificator care estimează probabilitatea ca un e-mail să fie spam, în funcție de cuvintele conținute de acesta. Se pornește de la o bază de date de antrenare care conține e-mailuri etichetate ca spam sau non-spam. Primul pas este specificarea caracteristicilor $X$ care descriu un e-mail. Se va reprezenta un e-mail printr-un vector de caracteristici a cărui lungime este egală cu numărul de cuvinte dintr-un dicționar. Dacă un mail conține cuvântul de la indexul $j$ din dicționar, atunci se va seta $x_j := x_j + 1$.

Pentru acest exemplu, baza de date de antrenare **a fost deja preprocesată** în așa fel încât fiecare email marchează în vectorul de caracteristici numărul de apariții al fiecărui cuvânt din dicționarul folosit.
Exercițiul din acest laborator folosește un dicționar de 2500 de cuvinte pentru caracteristicile fiecărui e-mail în parte. Astfel, vectorul de caracteristici $X$ din baza de date de antrenare va avea următoarea formă: $(\text{nr_exemple} \times \text{dim. dicționar})$. Dacă vor fi folosite 700 de emailuri și un dicționar de 2500 de cuvinte pentru datele de antrenare, atunci dimensiunea vectorului de caracteristici $X$ va fi: $(700 \times 2500)$.

## Exemplu de procesare pentru un e-mail
Să zicem că dorim să procesăm următorul e-mail:


> Buy now quick! Save 100 dollars with our special deal. Buy 10 pieces and get one free.

Iar pentru acest exemplu, presupunem că folosim un dicționar care conține toate cuvintele din acest mail. Așadar vom avea vectorul de caracteristici pentru acest e-mail:
$$x = \begin{bmatrix}1 \\ \vdots \\ 2 \\ \vdots \\ 1 \\ \vdots\end{bmatrix} = \begin{bmatrix}\text{and} \\ \vdots \\ \text{buy} \\ \vdots \\ \text{with} \\ \vdots \end{bmatrix}$$

## Implementarea clasificatorului Naive Bayes

Probabilitatea pe care vrem să o estimăm este probabilitatea ca un mesaj e-mail să fie spam, dându-se cuvintele conținute în respectivul e-mail: $P(y=1|x)$:
$$P(y=1|x) = \frac{P(x|y=1) \cdot P(y=1)}{P(x)} = \frac{\prod_{j=1}^nP(x_j|y=1) \cdot P(y=1)}{\prod_{j=1}^nP(x_j|y=1) \cdot P(y=1) + \prod_{j=1}^nP(x_j|y=0) \cdot P(y=0)}$$
unde:
*   $P(y=1|x)$ este probabilitatea ca un e-mail să fie spam, dându-se cuvintele conținând cuvintele $x$.
*   $P(x_j|y=1)$ reprezintă probabilitatea ca un cuvânt $x_j$ din dicționar să aparțină unui mesaj spam.
*   $P(y=1)$ reprezintă probabilitatea ca un e-mail să fie spam.
*   $P(y=0)$ este probabilitatea ca un mesaj să nu fie spam.
*   $P(x_j)$ reprezintă probabilitatea apariției cuvântului $x_j$.

Se poate calcula probabilitatea de apariție unui cuvânt dându-se un e-mail spam:
$$P(x_j|y=1) = \frac{N_{x_j|y=1} + \alpha}{N_{y=1} + \alpha \cdot N_V}$$
unde:
*  $P(x_j|y=1)$ este probabilitatea de apariție a cuvântului $x_j$ în mesaje spam.
*  $N_{x_j|y=1}$ este numărul de apariții ale cuvântului $x_j$ din mesajele spam.
*  $\alpha$ este un parametru de netezire, denumit netezirea Laplace.
*  $N_{y=1}$ este numărul total de cuvinte din toate mesajele spam.
*  $N_V$ este numărul total de cuvinte din vocabular.

Analog se poate calcula probabilitatea de apariție a aceluiași cuvânt într-un mesaj non-spam:
$$P(x_j|y=0) = \frac{N_{x_j|y=0} + \alpha}{N_{y=0} + \alpha \cdot N_V}$$

Prima oară se calculează constantele:


*   $N_{y=1}$ - numărul de mesaje spam.
*   $N_V$ - dimensiunea vocabularului.
*   $N_{y=0}$ - numărul de mesaje non-spam.
*   $P(y=1)$ - probabilitatea ca un e-mail (din baza de date) să fie spam.

In [ ]:
import numpy as np


# Incarcarea bazei de date de antrenare
x_train = np.loadtxt('train-features-full.txt', delimiter=' ')
y_train = np.loadtxt('train-labels.txt', delimiter=' ')

# Stocarea numarului de exemple de antrenare
num_train = x_train.shape[0]

# Gasirea indecsilor pentru etichetele spam, respectiv non-spam
spam_indices = np.nonzero(y_train==1)[0]
nonspam_indices = np.nonzero(y_train==0)[0]

N_Y1 = np.sum(x_train[spam_indices])
print(f"Numarul total de cuvinte din mailurile spam este: {N_Y1}")

N_V = x_train[0].shape[0]
print(f"In baza de date de antrenare se afla {num_train} emailuri. Sunt cunoscute {N_V} de cuvinte distincte.")

N_Y0 = np.sum(x_train[nonspam_indices])
print(f"Numarul total de cuvinte din mailurile non-spam este: {N_Y0}")

# Factorul de netezire Laplace
alpha = 1.0
print(f"Factorul de netezire este: {alpha}")

P_Y1 = spam_indices.shape[0] / num_train
P_Y0 = nonspam_indices.shape[0] / num_train

print(f"Probabilitatea ca un e-mail din baza de date sa fie spam este de {P_Y1}")
print(f"Probabilitatea ca un e-mail din baza de date sa fie non-spam este de {P_Y0}")


Numarul total de cuvinte din mailurile spam este: 91566.0
In baza de date de antrenare se afla 700 emailuri. Sunt cunoscute 2500 de cuvinte distincte.
Numarul total de cuvinte din mailurile non-spam este: 61762.0
Factorul de netezire este: 1.0
Probabilitatea ca un e-mail din baza de date sa fie spam este de 0.5
Probabilitatea ca un e-mail din baza de date sa fie non-spam este de 0.5


Apoi se calculează pentru fiecare cuvânt din dicționar $P(x_j|y=1)$, respectiv $P(x_j|y=0)$.

In [ ]:
P_spam = (np.sum(x_train[spam_indices], axis=0) + alpha) / (N_Y1 + alpha * N_V)
P_nonspam = (np.sum(x_train[nonspam_indices], axis=0) + alpha) / (N_Y0 + alpha * N_V)

Apoi se verifică din baza de date de test fiecare e-mail în parte. Pentru determinarea probabilității ca un e-mail să fie spam sau nonspam nu este necesar calculul numitorului din ecuația $P(y=1|x)$. De asemenea, pentru eficientizarea calculului, se aplică logaritmul peste densitățile de probabilitate $P(x_j|y=1)$, respectiv $P(x_j|y=0)$:

$$log(P(y=1|x)) \propto \sum_{j=1}^n\text{log}(P(x_j|y=1)) + \text{log}(P(y=1))$$
respectiv
$$log(P(y=0|x)) \propto \sum_{j=1}^n\text{log}(P(x_j|y=0)) + \text{log}(P(y=0))$$
unde simbolul $\propto$ reprezintă proporționalitatea.



In [ ]:
x_test = np.loadtxt('test-features-full.txt', delimiter=' ')
y_test = np.loadtxt('test-labels.txt', delimiter=' ')

P_spam_messages = np.dot(x_test, np.log(P_spam)) + np.log(P_Y1)
P_nonspam_messages = np.dot(x_test, np.log(P_nonspam)) + np.log(1. - P_Y1)


După ce au fost calculate probabilitățile ca e-mailurile din baza de date de test să fie spam și nonspam, se compară rezultatul algoritmului Naive Bayes cu valorile țintă ale setului de date de test.

In [ ]:
# Variabila res contine pentru fiecare email din baza de date de test True daca
# acel email este spam, respectiv False in caz contrar.
res = P_spam_messages > P_nonspam_messages

# Se numara documentele clasificate gresit
y_test = y_test.astype(bool)
num_wrong = res != y_test

print(f"Din emailurile stocate in baza de date de test au fost clasificate gresit \
{np.count_nonzero(num_wrong)} emailuri")
print(f"Eroarea statistica este de {np.count_nonzero(num_wrong) / y_test.shape[0]:.2f}")


Din emailurile stocate in baza de date de test au fost clasificate gresit 5 emailuri
Eroarea statistica este de 0.02


# Exerciții propuse

Fișierul SMSSpamCollection conține 5574 de mesaje SMS dintr-o bază de date, etichetate ca spam sau non-spam.

1.  Să se citească fișierul SMSSpamCollection și să se curețe fiecare mesaj în parte în felul următor: caracterele non-alfanumerice, respectiv cifrele să fie eliminate. Să se treacă toate mesajele în lowercase.
2.  Să se elimine cuvintele de legătură din fiecare mesaj.
3.  Să se definească un vocabular pe baza fiecărui cuvânt din mesajele din baza de date (vocabularul conține fiecare cuvânt unic din baza de date).
4.  Să se definească baza de date de antrenare în mod similar cu cea din exemplul anterior (fiecare cuvânt dintr-un mesaj incrementează valoarea din dicționarul folosit pentru variabila $x$; pentru $y$, se va considera pentru fiecare mesaj în parte $y=1$ dacă mesajul este considerat spam, respectiv $y=0$ în caz contrar). Baza de date de antrenare va conține primele 5000 de exemple $(x,y)$. Restul datelor vor fi folosite pentru testarea algoritmului.
5.  Să se clasifice fiecare mesaj din baza de date de test în spam sau nonspam.




## Observații
Pentru a elimina din mesaje caracterele non-alfanumerice și cifrele se folosesc **regular expressions** (regex). Acestea se folosesc cu ajutorul modulului **re**:

```
import re
```
Pentru a elimina cuvintele de legătură, se folosește modulul **nltk**. Un exemplu de utilizare este:


```
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

s = set(stopwords.words('english'))
```
Setul **s** din secțiunea de cod anterioară conține cuvintele din limba engleză considerate de legătură.




In [4]:
# Importul modulelor necesare
import numpy as np
import re
from collections import Counter

# Filtrarea textului mesajelor
def preprocess_text(text):
    """Curata si normalizeaza textul conform cerintelor:
    - elimina caractere non-alfanumerice si cifre
    - converteste la lowercase
    """
    # Converteste la lowercase
    text = text.lower()
    # Elimina cifrele si caracterele non-alfanumerice (pastreaza doar litere si spatii)
    text = re.sub(r'[^a-z\s]', '', text)
    # Elimina spatiile multiple
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Citirea fisierului si procesarea mesajelor
def load_sms_data(filepath='/content/sample_data/SMSSpamCollection'):
    """Incarca datele din fisier."""
    messages = []
    labels = []

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                label, message = parts
                labels.append(1 if label == 'spam' else 0)
                messages.append(preprocess_text(message))

    return messages, np.array(labels)

# Incarcarea datelor
messages, labels = load_sms_data()

print(f"Total SMS-uri incarcate: {len(messages)}")
print(f"Spam: {np.sum(labels)}, Ham: {len(labels) - np.sum(labels)}")

# Eliminarea cuvintelor de legatura: "a", "the", ...
# Definim manual setul de stop words (cuvinte de legatura)
stop_words = {
    'a', 'an', 'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for',
    'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'were', 'be',
    'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
    'would', 'could', 'should', 'may', 'might', 'must', 'can', 'i', 'you',
    'he', 'she', 'it', 'we', 'they', 'this', 'that', 'these', 'those',
    'my', 'your', 'his', 'her', 'its', 'our', 'their', 'me', 'him', 'us',
    'them', 'what', 'which', 'who', 'when', 'where', 'why', 'how', 'all',
    'each', 'every', 'both', 'few', 'more', 'most', 'other', 'some', 'such',
    'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
    'just', 'dont', 'now', 'get', 'got', 'go', 'goes', 'went'
}

def remove_stopwords(text):
    """Elimina cuvintele de legatura."""
    words = text.split()
    return ' '.join([word for word in words if word not in stop_words])

# Aplicarea eliminarii cuvintelor de legatura
messages = [remove_stopwords(msg) for msg in messages]

# Construirea vocabularului - fiecare cuvant unic din baza de date
all_words = []
for message in messages:
    all_words.extend(message.split())

# Construirea vocabularului (cuvinte care apar de cel putin 1 ori)
word_counts = Counter(all_words)
vocabulary = sorted(list(word_counts.keys()))  # vocabular sortat alfabetic

word_to_idx = {word: idx for idx, word in enumerate(vocabulary)}

print(f"Dimensiunea vocabularului: {len(vocabulary)}")

# Transformarea mesajelor din text in vocabular
def text_to_vector(text, word_to_idx):
    """Converteste un text intr-un vector de frecvente.
    Fiecare cuvant dintr-un mesaj incrementeaza valoarea din dictionar."""
    vector = np.zeros(len(word_to_idx))
    words = text.split()
    for word in words:
        if word in word_to_idx:
            vector[word_to_idx[word]] += 1
    return vector

# Construirea datelor de antrenare
# Primele 5000 de exemple pentru antrenare
X = np.array([text_to_vector(msg, word_to_idx) for msg in messages])
y = labels

# Definirea bazei de date de antrenare si testare
x_train = X[:5000]
y_train = y[:5000]

# Definirea x_test
x_test = X[5000:]

# Definirea y_test
y_test = y[5000:]

print(f"\nDimensiunea setului de antrenare: {x_train.shape}")
print(f"Dimensiunea setului de testare: {x_test.shape}")

'''
Rularea algoritmului Naive Bayes pentru exemplul cu SMS-uri spam/nonspam.
Codul prezentat anterior pentru exemplul cu email-uri ar trebui sa functioneze
si pentru acest exemplu cu SMS-uri.
'''
num_train = x_train.shape[0]

assert num_train >= 5000, "Numarul de exemple de antrenare nu coincide cu baza de date cu SMS-uri"

# Gasirea indecsilor pentru etichetele spam, respectiv non-spam
spam_indices = np.nonzero(y_train==1)[0]
nonspam_indices = np.nonzero(y_train==0)[0]

N_Y1 = np.sum(x_train[spam_indices])
print(f"\nNumarul total de cuvinte din SMS-urile spam este: {N_Y1}")

N_V = x_train[0].shape[0]
print(f"In baza de date de antrenare se afla {num_train} SMS-uri. Sunt cunoscute {N_V} de cuvinte distincte.")

N_Y0 = np.sum(x_train[nonspam_indices])
print(f"Numarul total de cuvinte din SMS-urile non-spam este: {N_Y0}")

# Factorul de netezire Laplace
alpha = 1.0
print(f"Factorul de netezire este: {alpha}")

P_Y1 = spam_indices.shape[0] / num_train
P_Y0 = nonspam_indices.shape[0] / num_train

print(f"Probabilitatea ca un SMS din baza de date sa fie spam este de {P_Y1}")
print(f"Probabilitatea ca un SMS din baza de date sa fie non-spam este de {P_Y0}")

P_spam = (np.sum(x_train[spam_indices], axis=0) + alpha) / (N_Y1 + alpha * N_V)
P_nonspam = (np.sum(x_train[nonspam_indices], axis=0) + alpha) / (N_Y0 + alpha * N_V)

P_spam_messages = np.dot(x_test, np.log(P_spam)) + np.log(P_Y1)
P_nonspam_messages = np.dot(x_test, np.log(P_nonspam)) + np.log(1. - P_Y1)

# Variabila res contine pentru fiecare SMS din baza de date de test True daca
# acel SMS este spam, respectiv False in caz contrar.
res = P_spam_messages > P_nonspam_messages

# Se numara documentele clasificate gresit
y_test = y_test.astype(bool)
num_wrong = res != y_test

print(f"\nDin SMS-urile stocate in baza de date de test au fost clasificate gresit \
{np.count_nonzero(num_wrong)} SMS-uri (din totalul de {y_test.shape[0]} SMS-uri).")
print(f"Eroarea statistica este de {np.count_nonzero(num_wrong) / y_test.shape[0]:.2f}")

Total SMS-uri incarcate: 5574
Spam: 747, Ham: 4827
Dimensiunea vocabularului: 8535

Dimensiunea setului de antrenare: (5000, 8535)
Dimensiunea setului de testare: (574, 8535)

Numarul total de cuvinte din SMS-urile spam este: 9821.0
In baza de date de antrenare se afla 5000 SMS-uri. Sunt cunoscute 8535 de cuvinte distincte.
Numarul total de cuvinte din SMS-urile non-spam este: 36412.0
Factorul de netezire este: 1.0
Probabilitatea ca un SMS din baza de date sa fie spam este de 0.1346
Probabilitatea ca un SMS din baza de date sa fie non-spam este de 0.8654

Din SMS-urile stocate in baza de date de test au fost clasificate gresit 15 SMS-uri (din totalul de 574 SMS-uri).
Eroarea statistica este de 0.03
